In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re 
wikipedia_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

print(f"Attempting to scrape data from: {wikipedia_url}")

try:
 
    response = requests.get(wikipedia_url)

    # Raise an HTTPError for bad responses (4xx or 5xx status codes)
    response.raise_for_status()

    html_content = response.text

    print("\nSuccessfully fetched HTML content.")
    soup = BeautifulSoup(html_content, 'html.parser')

    print("HTML content parsed with BeautifulSoup.")

    tables = soup.find_all('table', {'class': 'wikitable'})

    if not tables:
        print("\nNo tables with class 'wikitable' found on the page.")
    else:
        print(f"\nFound {len(tables)} tables with class 'wikitable'. Attempting to convert to DataFrames...")
        extracted_dfs = []

        for i, table in enumerate(tables):
            print(f"\n--- Processing Table {i+1} ---")
            try:
               
                df_list = pd.read_html(str(table))
                
                if df_list:
                   
                    df = df_list[0]
                    # --- Outcome: Display DataFrame Information for each table ---
                    print(f"Table {i+1} successfully converted to DataFrame.")
                    print(f"First 5 rows of DataFrame {i+1}:")
                    print(df.head())
                    print(f"\nDataFrame {i+1} Info:")
                    df.info()
                    extracted_dfs.append(df)
                else:
                    print(f"Could not convert Table {i+1} to DataFrame (df_list was empty).")
            except Exception as e:
                print(f"Error processing Table {i+1} with pandas.read_html: {e}")

        if extracted_dfs:
            print("\n--- Summary of Web Scraping Outcome ---")
            print(f"Successfully extracted and processed {len(extracted_dfs)} DataFrames from the Wikipedia page.")
            print("These DataFrames are stored in the 'extracted_dfs' list.")
        else:
            print("\nNo DataFrames were successfully extracted from the Wikipedia page.")

except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except requests.exceptions.ConnectionError as conn_err:
    print(f"Connection error occurred: {conn_err}")
except requests.exceptions.Timeout as timeout_err:
    print(f"Timeout error occurred: {timeout_err}")
except requests.exceptions.RequestException as req_err:
    print(f"An unexpected error occurred during the request: {req_err}")
except Exception as e:
    print(f"An unexpected error occurred during HTML parsing or DataFrame creation: {e}")

